In [ ]:
!pip install ultralytics 

In [ ]:
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir


In [1]:
from ultralytics import YOLO
import torch
import time
import cv2
import os
import shutil
from pathlib import Path
import pandas as pd

# -------------------------------
# Load trained model
# -------------------------------
model = YOLO("/kaggle/input/yolov8m/best.pt")  # path to your trained model

# -------------------------------
# Run evaluation on TEST SET
# -------------------------------
metrics = model.val(
    data="/kaggle/input/violence-weapon-detection/data.yaml",
    split="test",
    imgsz=640,
    save_json=True,
    conf=0.25,
    plots=True  # saves PR curve, F1 curve, confusion matrix
)

# -------------------------------
# Create output folder
# -------------------------------
save_dir = Path("/kaggle/working/eval_results")
save_dir.mkdir(parents=True, exist_ok=True)

# -------------------------------
# 1. QUANTITATIVE METRICS
# -------------------------------
results = {
    "Precision": metrics.box.p.mean(),   # mean precision across classes
    "Recall": metrics.box.r.mean(),      # mean recall across classes
    "F1_score": metrics.box.f1.mean(),   # mean F1 across classes
    "mAP50": metrics.box.map50           # overall mAP@0.5
}

# Per-class AP50
per_class_results = {}
for i, name in enumerate(model.names):
    _, _, ap50, _ = metrics.box.class_result(i)
    per_class_results[name] = ap50

# Save to CSV
df_main = pd.DataFrame([results])
df_main.to_csv(save_dir / "main_metrics.csv", index=False)

df_class = pd.DataFrame.from_dict(per_class_results, orient="index", columns=["AP50"])
df_class.to_csv(save_dir / "per_class_AP50.csv")

print("✅ Quantitative metrics saved!")

# -------------------------------
# 2. SAVE PLOTS (confusion matrix, PR curve, F1 curve)
# -------------------------------
plot_dir = Path(metrics.save_dir)
for file in plot_dir.glob("*.png"):
    shutil.copy(file, save_dir)

print("✅ Saved confusion matrix and curves!")

# -------------------------------
# 3. EFFICIENCY METRICS
# -------------------------------
model_size = os.path.getsize("/kaggle/input/yolov8m/best.pt") / (1024 * 1024)
params = sum(p.numel() for p in model.model.parameters())

try:
    flops = model.info(verbose=False)['flops']
except:
    flops = "Not supported"

# Inference speed test (first 50 test images)
test_imgs = list(Path("/kaggle/input/violence-weapon-detection/test/images").glob("*.jpg"))[:50]

start_time = time.time()
for img in test_imgs:
    model(img, imgsz=640, verbose=False)
end_time = time.time()

avg_time = (end_time - start_time) / len(test_imgs) * 1000  # ms per image
fps = 1000 / avg_time

efficiency = pd.DataFrame([{
    "Model size (MB)": model_size,
    "Parameters": params,
    "FLOPs": flops,
    "Inference time (ms/image)": avg_time,
    "FPS": fps
}])
efficiency.to_csv(save_dir / "efficiency_metrics.csv", index=False)

print("✅ Efficiency metrics saved!")

# -------------------------------
# 4. VISUAL RESULTS (detections)
# -------------------------------
vis_dir = save_dir / "detections"
vis_dir.mkdir(exist_ok=True)

example_imgs = test_imgs[:12]

for img_path in example_imgs:
    results_img = model(img_path, conf=0.25)
    results_img[0].save(filename=str(vis_dir / img_path.name))

print("✅ Detection examples saved!")

# -------------------------------
# 5. FAILURE CASES (only images with ground truth objects)
# -------------------------------
failure_dir = save_dir / "failure_cases"
failure_dir.mkdir(exist_ok=True)

labels_dir = Path("/kaggle/input/violence-weapon-detection/test/labels")

for img_path in test_imgs[:50]:
    # Corresponding label file
    label_path = labels_dir / (img_path.stem + ".txt")
    
    # Skip background images (no labels)
    if not label_path.exists() or os.path.getsize(label_path) == 0:
        continue
    
    # Run model
    r = model(img_path, conf=0.25, verbose=False)[0]

    # If model detected nothing → true failure
    if len(r.boxes) == 0:
        img = cv2.imread(str(img_path))
        cv2.imwrite(str(failure_dir / ("fail_" + img_path.name)), img)

print("✅ Failure cases saved! (background images ignored)")

print("\n🎉 All evaluation results saved in:", save_dir)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 7.0±1.6 MB/s, size: 41.0 KB)
val: Scanning /kaggle/input/violence-weapon-detection/test/labels... 937 images, 236 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 937/937 205.5it/s 4.6s0.0s
WARNING ⚠️ val: Cache directory /kaggle/input/violence-weapon-detection/test is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 59/59 2.5it/s 23.3s0.4ss
                   all        937        737   

In [3]:
import shutil

# Path to the folder you want to zip
folder_path = "/kaggle/working/runs"

# Output zip file path
zip_path = "/kaggle/working/eval_results_yolov8m_run.zip"

# Create zip
shutil.make_archive(base_name=zip_path.replace(".zip", ""), format='zip', root_dir=folder_path)

print(f"✅ Folder zipped successfully: {zip_path}")


✅ Folder zipped successfully: /kaggle/working/eval_results_yolov8m_run.zip
